In [1]:
import polars as pl
import datetime as dt
import numpy as np
import dask.dataframe as dd
from pyspark.sql import SparkSession
from pyspark.sql.functions import (to_timestamp, year, month, col, avg, month, dayofmonth, hour, minute, second, date_format, to_date, sum as _sum)
from pyspark.sql.functions import when, udf
from pyspark.sql.types import StringType

In [2]:
spark=SparkSession.builder.appName("Cabanuelas1").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/24 14:10:11 WARN Utils: Your hostname, Anduril, resolves to a loopback address: 127.0.1.1; using 172.16.81.227 instead (on interface wlo1)
25/07/24 14:10:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/24 14:10:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/24 14:10:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
var="TEMP"
T=spark.read.csv("IDEAM_Temperatura.csv",header=True,inferSchema=True)
meth=avg
#var="HUMI"
#T=spark.read.csv("IDEAM_Humedad_del_Aire_2_metros.csv",header=True,inferSchema=True)
#meth=avg
#var="RAIN"
#T=spark.read.csv("IDEAM_Precipitacion.csv",header=True,inferSchema=True)
#meth=_sum

In [4]:
T.printSchema()

root
 |-- CodigoEstacion: long (nullable = true)
 |-- CodigoSensor: integer (nullable = true)
 |-- FechaObservacion: string (nullable = true)
 |-- ValorObservado: double (nullable = true)
 |-- NombreEstacion: string (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- ZonaHidrografica: string (nullable = true)
 |-- Latitud: double (nullable = true)
 |-- Longitud: double (nullable = true)
 |-- DescripcionSensor: string (nullable = true)
 |-- UnidadMedida: string (nullable = true)



In [5]:
T.show(5)

+--------------+------------+--------------------+--------------+--------------------+------------+-----------+--------------------+-----------+------------+-----------------+------------+
|CodigoEstacion|CodigoSensor|    FechaObservacion|ValorObservado|      NombreEstacion|Departamento|  Municipio|    ZonaHidrografica|    Latitud|    Longitud|DescripcionSensor|UnidadMedida|
+--------------+------------+--------------------+--------------+--------------------+------------+-----------+--------------------+-----------+------------+-----------------+------------+
|      25025280|          68|11/14/2013 11:00:...|          30.0|EL TESORO IDEAM  ...|       SUCRE|     MORROA|BAJO MAGDALENA- C...|9.357083333|   -75.28925|    Temp Aire 2 m|          °C|
|      24035370|          68|08/25/2013 11:00:...|          11.5|     EL ESPINO - AUT|      BOYACÁ|  EL ESPINO|            SOGAMOSO|6.506722222|-72.45266667|    Temp Aire 2 m|          °C|
|      21206920|          68|10/08/2018 02:00:...|     

In [6]:
map1={"SANTANDER":"SANTANDER","CÓRDOBA":"CORDOBA","NARIÑO":"NARINO","BOYACÁ":"BOYACA","CAQUETA":"CAQUETA","CASANARE":"CASANARE",
      "META":"META","AMAZONAS":"AMAZONAS","GUAINÍA":"GUAINIA","GUAVIARE":"GUAVIARE","CAUCA":"CAUCA","ANTIOQUIA":"ANTIOQUIA",
      "BOLÍVAR":"BOLIVAR","CHOCO":"CHOCO","CALDAS":"CALDAS","ATLÁNTICO":"ATLANTICO","VALLE DEL CAUCA":"VALLE DEL CAUCA",
      "CUNDINAMARCA":"CUNDINAMARCA","LA GUAJIRA":"LA GUAJIRA","NARINO":"NARINO","TOLIMA":"TOLIMA","ARAUCA":"ARAUCA",
      "NORTE DE SANTANDER":"NORTE DE SANTANDER","SAN ANDRÉS PROVIDENCIA":"SAN ANDRES PROVIDENCIA","BOGOTA":"BOGOTA","HUILA":"HUILA",
      "SUCRE":"SUCRE","ATLANTICO":"ATLANTICO","CHOCÓ":"CHOCO","BOLIVAR":"BOLIVAR","BOGOTA D.C.":"BOGOTA","RISARALDA":"RISARALDA","PUTUMAYO":"PUTUMAYO",
      "MAGDALENA":"MAGDALENA","QUINDÍO":"QUINDIO","CORDOBA":"CORDOBA","CESAR":"CESAR","VAUPÉS":"VAUPES","VICHADA":"VICHADA",
      "ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA":"SAN ANDRES PROVIDENCIA","BOGOTÁ":"BOGOTA","CAQUETÁ":"CAQUETA",
      "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA":"SAN ANDRES PROVIDENCIA","ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA":"SAN ANDRES PROVIDENCIA",
     "ARCHIPIÉLAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA":"SAN ANDRES PROVIDENCIA"}

mapping_bc=spark.sparkContext.broadcast(map1)
def map_col(x):
    return mapping_bc.value.get(x,x)

map_col_udf=udf(map_col,StringType())
T=T.withColumn("Departamento",map_col_udf("Departamento"))

In [7]:
# 1) Selecciono sólo las columnas que necesito
df1 = T.select(
    col("FechaObservacion"),
    col("Departamento"),
    col("ValorObservado")
)

# 2) Parseo FechaObservacion y la convierto a Timestamp 24 h
#    – to_timestamp(..., "MM/dd/yyyy hh:mm:ss a") hace el parseo AM/PM
#    – date_format(..., "yyyy-MM-dd HH:mm:ss") la formatea como string 24 h
df2 = df1.withColumn(
    "Fecha24h",
    date_format(
        to_timestamp(col("FechaObservacion"), "MM/dd/yyyy hh:mm:ss a"),
        "yyyy-MM-dd HH:mm:ss"
    )
).drop("FechaObservacion")  # opcional: elimino la columna original

# 3) Reordeno columnas si quieres:
df2 = df2.select("Fecha24h", "ValorObservado", "Departamento")

# 4) Compruebo
df2.printSchema()
df2.show(5, truncate=False)

root
 |-- Fecha24h: string (nullable = true)
 |-- ValorObservado: double (nullable = true)
 |-- Departamento: string (nullable = true)



[Stage 3:>                                                          (0 + 1) / 1]

+-------------------+--------------+------------+
|Fecha24h           |ValorObservado|Departamento|
+-------------------+--------------+------------+
|2013-11-14 11:00:00|30.0          |SUCRE       |
|2013-08-25 11:00:00|11.5          |BOYACA      |
|2018-10-08 14:00:00|10.5          |BOGOTA      |
|2019-01-21 10:54:00|10.54         |BOGOTA      |
|2016-05-15 11:40:00|15.1          |BOGOTA      |
+-------------------+--------------+------------+
only showing top 5 rows


In [8]:
# 2) Creamos columna de tipo Timestamp y luego columna de solo Date
df3 = (
    df2
    .withColumn(
        "ts",
        to_timestamp(col("Fecha24h"), "yyyy-MM-dd HH:mm:ss")
    )
    .withColumn(
        "date",
        to_date(col("ts"))      # extrae YYYY-MM-DD
    )
)

# 3) Agrupamos por Departamento y date, sumando ValorObservado
daily_dept = (
    df3
    .groupBy("Departamento", "date")
    .agg(
        meth(col("ValorObservado")).alias("sum_ValorObservado")
    )
    .orderBy("Departamento", "date")
)

In [9]:
daily_dept.write.option("header", "true").csv("{}_{}_ano_depto".format(meth,var))